# Imports

In [1]:
%run PII_Util.py

D:\Anaconda3\envs\nlp_torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('dsda')

dsda


# Load Data

In [3]:
path_train = './in/train.json'
path_test = './in/test.json'

In [4]:
train_json = json.load(open(path_train))
df_unsplitted = pd.json_normalize(train_json)

test_json = json.load(open(path_test))
df_test = pd.json_normalize(test_json)

# Split (Stratified)

In [10]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
def stratified_split(df_unsplitted, possible_labels, test_size=0.12, random_state=42):
    df_dist = label_count_columns(df_unsplitted, classes)
    df_dist = perform_dbscan_clustering(df_dist, eps=0.5, min_samples=5)

    df_train, df_valid = train_test_split(df_unsplitted, test_size=test_size, random_state=random_state, stratify=df_dist['cluster'])
    return df_train, df_valid

def label_count_columns(df, possible_labels):
    # Initialize a dictionary to store the count of each label
    label_counts = {label: [] for label in possible_labels}
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Use Counter to count the occurrence of each label in the sequence
        row_label_counts = Counter(row['labels'])
        
        # Append the counts to the label_counts dictionary
        for label in possible_labels:
            label_counts[label].append(row_label_counts[label])
    
    # Create a new DataFrame from the label_counts dictionary
    label_counts_df = pd.DataFrame(label_counts)
    
    return label_counts_df



def perform_dbscan_clustering(label_counts_df, eps=0.5, min_samples=5):
    # Standardize the feature values
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(label_counts_df)
    
    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan.fit(scaled_features)
    
    # Add cluster labels to the DataFrame
    label_counts_df['cluster'] = dbscan.labels_
    
    return label_counts_df

In [11]:
df_train, df_valid = stratified_split(df_unsplitted, classes, test_size=0.12, random_state=42)

# Preprocessing

In [15]:
from datasets import Dataset

In [16]:
ds_train = Dataset.from_pandas(df_train)
ds_valid = Dataset.from_pandas(df_valid)

In [14]:
%%time
model_adapter = Yanis_Adapter(threshold = 0.1, from_scratch=True, mode_bio='reorder')

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at Yanis/microsoft-deberta-v3-large_ner_conll2003-anonimization_TRY_1 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([18, 1024]) in the checkpoint and torch.Size([15, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([18]) in the checkpoint and torch.Size([15]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: total: 7.08 s
Wall time: 11.9 s


In [17]:
%%time
ds_prep_train = ds_train.map(preprocess_from_words, fn_kwargs={"model_adapter": model_adapter, "col_words": "tokens", "col_labels": "labels", "mode_bio": model_adapter.mode_bio}, batched=False)
ds_prep_valid = ds_valid.map(preprocess_from_words, fn_kwargs={"model_adapter": model_adapter, "col_words": "tokens", "col_labels": "labels", "mode_bio": model_adapter.mode_bio}, batched=False)

Map: 100%|███████████████████████████████████████████████████████████████████| 817/817 [00:07<00:00, 111.39 examples/s]

CPU times: total: 1min 12s
Wall time: 1min 14s


# Training

In [18]:
model = model_adapter.model

In [19]:
data_collator = DataCollatorForTokenClassification(model_adapter.tokenizer)

In [20]:
temp_train = ds_prep_train.select(range(0, 2))
temp_valid = ds_prep_valid.select(range(0, 2))

In [22]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

In [23]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=1000,
    output_dir='./results', 
    optim="adafactor",
    per_device_eval_batch_size=1,        # batch size for evaluation
    eval_accumulation_steps = 1,
    
#     warmup_steps=500,                    # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,                   # strength of weight decay
    evaluation_strategy="steps",
    eval_steps=1000,                      # evaluate every 500 steps
    load_best_model_at_end=True,           # stop training if no improvement for 3 evaluations
    
)

trainer = Trainer(
    model=model,                       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                # training arguments, defined above
    train_dataset=temp_train,       # training dataset
    eval_dataset=temp_valid,          # evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    data_collator=data_collator,

    
)

In [24]:
#Freeze Base Layers

for param in model.base_model.parameters():
    param.requires_grad = False

In [25]:
result = trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
pred_results = trainer.predict(dataset_filtered)

In [26]:
best_model_path = trainer.state.best_model_checkpoint

# Now you can save the best model wherever you want
# For example, you can use shutil to copy the best model to a desired directory
import shutil
shutil.copytree(best_model_path, "./mock/yanis_02")

TypeError: expected str, bytes or os.PathLike object, not NoneType